In [41]:
import pandas as pd
import numpy as np

In [42]:
clust_teams = pd.read_csv(r'C:\Users\krist\Documents\BME\2024_2025_1\Diplomatervezes_2\data\ML_outputs\team_clusters.csv')
#clust_players = pd.read_csv(r'C:\Users\krist\Documents\BME\2024_2025_1\Diplomatervezes_2\data\ML_outputs\player_clusters.csv')

clust_players = pd.read_csv(r'C:\Users\krist\Documents\BME\2024_2025_1\Diplomatervezes_2\data\player_stats_clustered.csv')



In [43]:
clust_players.head()

,Rk_x,Player,Pos_x,Age_x,Tm,G_x,GS,MP_x,FG,FGA,...,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Player-additional_y,Cluster
0,1,Quincy Acy,SF,23,TOR,7,0,8.7,0.9,2.0,...,0.1,0.1,0.2,0.188,1.2,3.7,5.0,0.1,acyqu01,4
1,1,Quincy Acy,SF,23,SAC,56,0,14.0,1.1,2.3,...,0.5,0.7,1.3,0.078,-2.8,0.6,-2.2,0.0,acyqu01,4
2,2,Steven Adams,C,20,OKC,81,20,14.8,1.1,2.3,...,0.9,1.9,2.9,0.114,-3.1,0.9,-2.2,-0.1,adamsst01,4
3,3,Jeff Adrien,PF,27,CHA,25,0,10.2,0.9,1.6,...,0.4,0.5,0.9,0.170,-1.8,1.7,0.0,0.1,adrieje01,4
4,3,Jeff Adrien,PF,27,MIL,28,12,25.2,4.3,8.4,...,1.2,0.6,1.8,0.121,0.6,-1.8,-1.2,0.1,adrieje01,2


In [44]:
list(clust_players.columns)

['Rk_x',
 'Player',
 'Pos_x',
 'Age_x',
 'Tm',
 'G_x',
 'GS',
 'MP_x',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 '2P',
 '2PA',
 '2P%',
 'eFG%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'Player-additional_x',
 'Season',
 'Rk_y',
 'Pos_y',
 'Age_y',
 'G_y',
 'MP_y',
 'PER',
 'TS%',
 '3PAr',
 'FTr',
 'ORB%',
 'DRB%',
 'TRB%',
 'AST%',
 'STL%',
 'BLK%',
 'TOV%',
 'USG%',
 'OWS',
 'DWS',
 'WS',
 'WS/48',
 'OBPM',
 'DBPM',
 'BPM',
 'VORP',
 'Player-additional_y',
 'Cluster']

In [45]:
rookies = [
    "Paolo Banchero", "Chet Holmgren", "Jabari Smith Jr.", "Keegan Murray", "Jaden Ivey", 
    "Bennedict Mathurin", "Shaedon Sharpe", "Dyson Daniels", "Jeremy Sochan", "Johnny Davis", 
    "Ousmane Dieng", "Jalen Williams", "Jalen Duren", "Ochai Agbaji", "Mark Williams", 
    "AJ Griffin", "Tari Eason", "Dalen Terry", "Jake LaRavia", "Malaki Branham", 
    "Christian Braun", "Walker Kessler", "David Roddy", "MarJon Beauchamp", "Blake Wesley", 
    "Wendell Moore Jr.", "Nikola Jović", "Patrick Baldwin Jr.", "TyTy Washington Jr.", "Peyton Watson"
]

clust_players['isRookie'] = clust_players['Player'].apply(lambda x: 1 if x in rookies else 0)
clust_players.head()

,Rk_x,Player,Pos_x,Age_x,Tm,G_x,GS,MP_x,FG,FGA,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Player-additional_y,Cluster,isRookie
0,1,Quincy Acy,SF,23,TOR,7,0,8.7,0.9,2.0,...,0.1,0.2,0.188,1.2,3.7,5.0,0.1,acyqu01,4,0
1,1,Quincy Acy,SF,23,SAC,56,0,14.0,1.1,2.3,...,0.7,1.3,0.078,-2.8,0.6,-2.2,0.0,acyqu01,4,0
2,2,Steven Adams,C,20,OKC,81,20,14.8,1.1,2.3,...,1.9,2.9,0.114,-3.1,0.9,-2.2,-0.1,adamsst01,4,0
3,3,Jeff Adrien,PF,27,CHA,25,0,10.2,0.9,1.6,...,0.5,0.9,0.170,-1.8,1.7,0.0,0.1,adrieje01,4,0
4,3,Jeff Adrien,PF,27,MIL,28,12,25.2,4.3,8.4,...,0.6,1.8,0.121,0.6,-1.8,-1.2,0.1,adrieje01,2,0


In [46]:
non_rookies = clust_players[clust_players['isRookie'] == 0]
guards = non_rookies[non_rookies['Pos_x'].isin(['PG', 'SG'])]
forwards = non_rookies[non_rookies['Pos_x'].isin(['SF', 'PF'])]
centers = non_rookies[non_rookies['Pos_x'] == 'C']


In [47]:

# Aggregation for guards
guards_agg = guards.groupby(['Season','Tm', 'Cluster']).size().unstack(fill_value=0)
guards_agg.columns = [f'Guard_Cluster_{col}' for col in guards_agg.columns]

# Aggregation for forwards
forwards_agg = forwards.groupby(['Season','Tm', 'Cluster']).size().unstack(fill_value=0)
forwards_agg.columns = [f'Forward_Cluster_{col}' for col in forwards_agg.columns]

# Aggregation for centers
centers_agg = centers.groupby(['Season','Tm', 'Cluster']).size().unstack(fill_value=0)
centers_agg.columns = [f'Center_Cluster_{col}' for col in centers_agg.columns]

# Merge all aggregations
team_agg = guards_agg.join(forwards_agg, how='outer').join(centers_agg, how='outer').fillna(0).astype(int)
(team_agg)

Guard_Cluster_1  Guard_Cluster_2  Guard_Cluster_3  \
Season  Tm                                                       
2013-14 ATL                0                0                2   
        BOS                0                0                3   
        BRK                0                0                3   
        CHA                0                0                2   
        CHI                0                0                4   
...                      ...              ...              ...   
2022-23 SAC                1                0                2   
        SAS                0                0                3   
        TOR                1                0                1   
        UTA                0                0                6   
        WAS                1                0                1   

             Guard_Cluster_4  Guard_Cluster_5  Guard_Cluster_6  \
Season  Tm                                                       
2013-14 ATL                0                2                3   
        BOS                0                3                4   
        BRK                0                2                3   
        CHA                0                3                2   
        CHI                0                1                3   
...                      ...              ...              ...   
2022-23 SAC                0                2                2   
        SAS                0                3                0   
        TOR                0                2                2   
        UTA                0                1                2   
        WAS                0                4                3   

             Forward_Cluster_1  Forward_Cluster_2  Forward_Cluster_3  \
Season  Tm                                                             
2013-14 ATL                  1                  0                  1   
        BOS                  0                  3                  1   
        BRK                  0                  1                  1   
        CHA                  0                  1                  1   
        CHI                  0                  2                  2   
...                        ...                ...                ...   
2022-23 SAC                  0                  0                  1   
        SAS                  0                  0                  1   
        TOR                  1                  2                  1   
        UTA                  1                  1                  0   
        WAS                  0                  0                  2   

             Forward_Cluster_4  Forward_Cluster_5  Forward_Cluster_6  \
Season  Tm                                                             
2013-14 ATL                  1                  2                  1   
        BOS                  0                  2                  0   
        BRK                  1                  3                  1   
        CHA                  1                  3                  2   
        CHI                  0                  0                  5   
...                        ...                ...                ...   
2022-23 SAC                  1                  1                  5   
        SAS                  1                  4                  3   
        TOR                  2                  1                  3   
        UTA                  2                  3                  2   
        WAS                  1                  3                  2   

             Center_Cluster_1  Center_Cluster_2  Center_Cluster_3  \
Season  Tm                                                          
2013-14 ATL                 0                 1                 0   
        BOS                 0                 0                 0   
        BRK                 0                 1                 0   
        CHA                 0                 1                 0   
       

In [48]:
#merge the team_agg to the clust_players
players_all = clust_players.merge(team_agg, left_on=['Season', 'Tm'], right_on=['Season', 'Tm'], how='left')

players_all.head()

,Rk_x,Player,Pos_x,Age_x,Tm,G_x,GS,MP_x,FG,FGA,...,Forward_Cluster_3,Forward_Cluster_4,Forward_Cluster_5,Forward_Cluster_6,Center_Cluster_1,Center_Cluster_2,Center_Cluster_3,Center_Cluster_4,Center_Cluster_5,Center_Cluster_6
0,1,Quincy Acy,SF,23,TOR,7,0,8.7,0.9,2.0,...,1,3,0,4,0,1,0,2,0,0
1,1,Quincy Acy,SF,23,SAC,56,0,14.0,1.1,2.3,...,0,4,4,1,1,0,0,2,0,1
2,2,Steven Adams,C,20,OKC,81,20,14.8,1.1,2.3,...,0,1,1,3,0,0,0,2,0,1
3,3,Jeff Adrien,PF,27,CHA,25,0,10.2,0.9,1.6,...,1,1,3,2,0,1,0,2,0,1
4,3,Jeff Adrien,PF,27,MIL,28,12,25.2,4.3,8.4,...,2,2,2,1,0,2,0,1,0,0


In [49]:
#remove the rows where the isRookie is 1
players_all_nr = players_all[players_all['isRookie'] == 0]
rookies = players_all[players_all['isRookie'] == 1]

In [50]:
#keep only the columns that are needed which are: Player, Pos_x, Age_x, Cluster, Guard_Cluster_1, Guard_Cluster_2, Guard_Cluster_3, Guard_Cluster_4, Guard_Cluster_5, Guard_Cluster_6, Forward_Cluster_1, Forward_Cluster_2, Forward_Cluster_3, Forward_Cluster_4, Forward_Cluster_5, Forward_Cluster_6, Center_Cluster_1, Center_Cluster_2, Center_Cluster_3, Center_Cluster_4, Center_Cluster_5, Center_Cluster_6
players_all_tr = players_all_nr[[ 'Pos_x', 'Age_x', 'Cluster', 'Guard_Cluster_1', 'Guard_Cluster_2', 'Guard_Cluster_3', 'Guard_Cluster_4', 'Guard_Cluster_5', 'Guard_Cluster_6', 'Forward_Cluster_1', 'Forward_Cluster_2', 'Forward_Cluster_3', 'Forward_Cluster_4', 'Forward_Cluster_5', 'Forward_Cluster_6', 'Center_Cluster_1', 'Center_Cluster_2', 'Center_Cluster_3', 'Center_Cluster_4', 'Center_Cluster_5', 'Center_Cluster_6']]
#map the positions to numbers
players_all_tr['Pos_x'] = players_all_tr['Pos_x'].map({'PG': 1, 'SG': 2, 'SF': 3, 'PF': 4, 'C': 5})

players_all_tr.head()

C:\Users\krist\AppData\Local\Temp\ipykernel_35616\3097870297.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Pos_x,Age_x,Cluster,Guard_Cluster_1,Guard_Cluster_2,Guard_Cluster_3,Guard_Cluster_4,Guard_Cluster_5,Guard_Cluster_6,Forward_Cluster_1,...,Forward_Cluster_3,Forward_Cluster_4,Forward_Cluster_5,Forward_Cluster_6,Center_Cluster_1,Center_Cluster_2,Center_Cluster_3,Center_Cluster_4,Center_Cluster_5,Center_Cluster_6
0,3,23,4,2,0,1,0,2,3,0,...,1,3,0,4,0,1,0,2,0,0
1,3,23,4,1,0,0,0,5,2,1,...,0,4,4,1,1,0,0,2,0,1
2,5,20,4,1,0,1,1,3,2,1,...,0,1,1,3,0,0,0,2,0,1
3,4,27,4,0,0,2,0,3,2,0,...,1,1,3,2,0,1,0,2,0,1
4,4,27,2,0,0,2,1,4,0,0,...,2,2,2,1,0,2,0,1,0,0


In [51]:
rookies.head()

,Rk_x,Player,Pos_x,Age_x,Tm,G_x,GS,MP_x,FG,FGA,...,Forward_Cluster_3,Forward_Cluster_4,Forward_Cluster_5,Forward_Cluster_6,Center_Cluster_1,Center_Cluster_2,Center_Cluster_3,Center_Cluster_4,Center_Cluster_5,Center_Cluster_6
5355,4,Ochai Agbaji,SG,22,UTA,59,22,20.5,2.8,6.5,...,0,2,3,2,0,0,1,2,0,0
5373,20,Patrick Baldwin Jr.,SF,20,GSW,31,0,7.3,1.4,3.5,...,2,1,2,1,0,1,0,1,0,0
5377,23,Paolo Banchero,PF,20,ORL,72,72,33.8,6.7,15.6,...,1,1,1,2,0,1,0,2,1,0
5394,37,MarJon Beauchamp,SF,22,MIL,52,11,13.5,1.8,4.7,...,1,0,5,1,0,1,0,2,0,1
5416,55,Malaki Branham,SG,19,SAS,66,32,23.5,4.1,9.3,...,1,1,4,3,0,2,0,3,0,0


In [52]:
#only keep the Player, Pos_x, Age_x, Cluster columns
rookies = rookies[['Player','Pos_x', 'Age_x', 'Cluster']]
#make the Player column the index
rookies.set_index('Player', inplace=True)
#map the positions to numbers
rookies['Pos_x'] = rookies['Pos_x'].map({'PG': 1, 'SG': 2, 'SF': 3, 'PF': 4, 'C': 5})
#get the team_agg for the 22-23 season
team_agg_22_23 = team_agg[team_agg.index.get_level_values('Season') == '2022-23']
#make the index the Tm column
team_agg_22_23.reset_index(inplace=True)
team_agg_22_23.set_index('Tm', inplace=True)
#drop the Season column
team_agg_22_23.drop(columns='Season', inplace=True)


C:\Users\krist\AppData\Local\Temp\ipykernel_35616\3752398557.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [53]:
team_agg_22_23.head()

,Guard_Cluster_1,Guard_Cluster_2,Guard_Cluster_3,Guard_Cluster_4,Guard_Cluster_5,Guard_Cluster_6,Forward_Cluster_1,Forward_Cluster_2,Forward_Cluster_3,Forward_Cluster_4,Forward_Cluster_5,Forward_Cluster_6,Center_Cluster_1,Center_Cluster_2,Center_Cluster_3,Center_Cluster_4,Center_Cluster_5,Center_Cluster_6
Tm,,,,,,,,,,,,,,,,,,
ATL,1,0,2,0,1,5,0,1,1,2,1,1,0,2,0,1,0,1
BOS,0,0,3,0,1,1,2,0,0,0,2,3,0,1,1,3,1,0
BRK,1,0,3,0,4,3,1,0,3,0,4,2,0,1,0,1,0,2
CHI,1,0,0,0,5,1,1,0,1,3,0,1,0,1,0,2,0,1
CHO,0,0,2,0,2,3,0,0,4,0,2,2,0,2,0,1,0,0



rookie = rookies.loc['Paolo Banchero']
team = team_agg_22_23.loc['LAL']

rookie_team = pd.concat([rookie, team])
rookie_team = rookie_team.to_frame().T

pts_pred = model.predict(rookie_team)
pts_pred

In [54]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [55]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

#PTS
X = players_all_tr
y = players_all_nr['PTS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models
models = {
    'RandomForest': RandomForestRegressor(),
    'LinearRegression': LinearRegression(),
    'DecisionTree': DecisionTreeRegressor(),
    'CatBoost': CatBoostRegressor(verbose=0),
    'XGBoost': XGBRegressor(),
}

# Train and evaluate each model
best_model = None
best_score = float('-inf')

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    print(f'{name} R2: {r2}')
    
    if r2 > best_score:
        best_score = r2
        best_model = model

model_pts = best_model
model_pts.fit(X_train, y_train)

y_pred = model_pts.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R2: {r2}')

RandomForest R2: 0.8011925608865592
LinearRegression R2: 0.7240167863945313
DecisionTree R2: 0.6270464031923002
CatBoost R2: 0.8132911670581638
XGBoost R2: 0.7886136819388067
MSE: 7.23087230656352
MAE: 2.0071215894797665
R2: 0.8132911670581638


In [56]:
from sklearn.model_selection import GridSearchCV

#TRB
# Define the parameter grid
param_grid = {
'iterations': [100, 200],
    'learning_rate': [0.01, 0.1],
    'depth': [3, 6]
}

# Initialize the model
cb = CatBoostRegressor()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=cb, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best model
best_rf = grid_search.best_estimator_

# Use the best model to predict
y_pred = best_rf.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Best Parameters: {grid_search.best_params_}')
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R2: {r2}')


Fitting 3 folds for each of 8 candidates, totalling 24 fits
0:	learn: 5.6279637	total: 970us	remaining: 193ms
1:	learn: 5.2094595	total: 1.73ms	remaining: 172ms
2:	learn: 4.8435033	total: 2.42ms	remaining: 159ms
3:	learn: 4.5249720	total: 3.11ms	remaining: 152ms
4:	learn: 4.2490932	total: 3.79ms	remaining: 148ms
5:	learn: 4.0371528	total: 4.41ms	remaining: 142ms
6:	learn: 3.8297395	total: 5.19ms	remaining: 143ms
7:	learn: 3.6528590	total: 5.98ms	remaining: 144ms
8:	learn: 3.5028304	total: 6.74ms	remaining: 143ms
9:	learn: 3.3853279	total: 7.44ms	remaining: 141ms
10:	learn: 3.2759689	total: 8.12ms	remaining: 140ms
11:	learn: 3.1845082	total: 8.79ms	remaining: 138ms
12:	learn: 3.1185961	total: 9.39ms	remaining: 135ms
13:	learn: 3.0437516	total: 10ms	remaining: 133ms
14:	learn: 2.9847409	total: 10.6ms	remaining: 131ms
15:	learn: 2.9370098	total: 11.2ms	remaining: 129ms
16:	learn: 2.8961849	total: 12ms	remaining: 129ms
17:	learn: 2.8548357	total: 12.7ms	remaining: 128ms
18:	learn: 2.823913

In [57]:
from sklearn.model_selection import GridSearchCV

#TRB
# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the model
rf = RandomForestRegressor()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best model
best_rf = grid_search.best_estimator_

# Use the best model to predict
y_pred = best_rf.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Best Parameters: {grid_search.best_params_}')
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R2: {r2}')


Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}
MSE: 6.9243771848335145
MAE: 1.9675263406266204
R2: 0.821205197904571


In [58]:
#PTS
X = players_all_tr
y = players_all_nr['PTS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_pts = CatBoostRegressor(depth=3, iterations=200, learning_rate=0.1) #({'depth': 3, 'iterations': 200, 'learning_rate': 0.1})
#RandomForestRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=100)
model_pts.fit(X_train, y_train)

y_pred = model_pts.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R2: {r2}')


0:	learn: 5.6279637	total: 1.05ms	remaining: 210ms
1:	learn: 5.2094595	total: 2.2ms	remaining: 217ms
2:	learn: 4.8435033	total: 3.03ms	remaining: 199ms
3:	learn: 4.5249720	total: 3.85ms	remaining: 189ms
4:	learn: 4.2490932	total: 4.78ms	remaining: 186ms
5:	learn: 4.0371528	total: 5.51ms	remaining: 178ms
6:	learn: 3.8297395	total: 6.54ms	remaining: 180ms
7:	learn: 3.6528590	total: 7.34ms	remaining: 176ms
8:	learn: 3.5028304	total: 8.18ms	remaining: 173ms
9:	learn: 3.3853279	total: 9.14ms	remaining: 174ms
10:	learn: 3.2759689	total: 9.91ms	remaining: 170ms
11:	learn: 3.1845082	total: 10.7ms	remaining: 168ms
12:	learn: 3.1185961	total: 11.6ms	remaining: 167ms
13:	learn: 3.0437516	total: 12.5ms	remaining: 166ms
14:	learn: 2.9847409	total: 13.3ms	remaining: 164ms
15:	learn: 2.9370098	total: 14.1ms	remaining: 162ms
16:	learn: 2.8961849	total: 15ms	remaining: 162ms
17:	learn: 2.8548357	total: 16.1ms	remaining: 163ms
18:	learn: 2.8239133	total: 17ms	remaining: 162ms
19:	learn: 2.8016814	total:

In [59]:
#TRB
X = players_all_tr
y = players_all_nr['TRB']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_trb = CatBoostRegressor(depth=3, iterations=200, learning_rate=0.1)
#RandomForestRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=100)
model_trb.fit(X_train, y_train)

y_pred = model_trb.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R2: {r2}')


0:	learn: 2.2569583	total: 1.07ms	remaining: 213ms
1:	learn: 2.1226781	total: 1.93ms	remaining: 191ms
2:	learn: 2.0072399	total: 2.54ms	remaining: 167ms
3:	learn: 1.9085408	total: 3.34ms	remaining: 163ms
4:	learn: 1.8158267	total: 4.15ms	remaining: 162ms
5:	learn: 1.7359528	total: 5.15ms	remaining: 167ms
6:	learn: 1.6577649	total: 5.88ms	remaining: 162ms
7:	learn: 1.6069418	total: 6.66ms	remaining: 160ms
8:	learn: 1.5541186	total: 7.46ms	remaining: 158ms
9:	learn: 1.5072873	total: 8.52ms	remaining: 162ms
10:	learn: 1.4687792	total: 9.28ms	remaining: 159ms
11:	learn: 1.4298888	total: 9.99ms	remaining: 156ms
12:	learn: 1.3998937	total: 10.7ms	remaining: 154ms
13:	learn: 1.3725267	total: 11.5ms	remaining: 153ms
14:	learn: 1.3511076	total: 12.3ms	remaining: 151ms
15:	learn: 1.3316371	total: 12.9ms	remaining: 149ms
16:	learn: 1.3170106	total: 13.7ms	remaining: 147ms
17:	learn: 1.3052364	total: 14.4ms	remaining: 145ms
18:	learn: 1.2963845	total: 15.4ms	remaining: 146ms
19:	learn: 1.2874891	t

In [60]:
#AST
X = players_all_tr
y = players_all_nr['AST']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_ast = CatBoostRegressor(depth=3, iterations=200, learning_rate=0.1)
#RandomForestRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=100)
model_ast.fit(X_train, y_train)

y_pred = model_ast.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R2: {r2}')


0:	learn: 1.6712134	total: 1.5ms	remaining: 300ms
1:	learn: 1.5714629	total: 3.3ms	remaining: 326ms
2:	learn: 1.4854442	total: 4.62ms	remaining: 303ms
3:	learn: 1.4116589	total: 5.68ms	remaining: 278ms
4:	learn: 1.3500349	total: 6.75ms	remaining: 263ms
5:	learn: 1.2958096	total: 7.98ms	remaining: 258ms
6:	learn: 1.2443994	total: 8.83ms	remaining: 243ms
7:	learn: 1.1989131	total: 9.5ms	remaining: 228ms
8:	learn: 1.1662863	total: 10.2ms	remaining: 216ms
9:	learn: 1.1341175	total: 10.8ms	remaining: 206ms
10:	learn: 1.1061927	total: 11.6ms	remaining: 199ms
11:	learn: 1.0792232	total: 12.2ms	remaining: 191ms
12:	learn: 1.0567952	total: 12.9ms	remaining: 186ms
13:	learn: 1.0403011	total: 13.6ms	remaining: 181ms
14:	learn: 1.0274919	total: 14.3ms	remaining: 176ms
15:	learn: 1.0150984	total: 14.9ms	remaining: 172ms
16:	learn: 1.0008221	total: 15.6ms	remaining: 168ms
17:	learn: 0.9894878	total: 16.2ms	remaining: 164ms
18:	learn: 0.9804311	total: 16.9ms	remaining: 161ms
19:	learn: 0.9728634	tota

In [61]:
#PER
X = players_all_tr
y = players_all_nr['PER']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_per = CatBoostRegressor(depth=3, iterations=200, learning_rate=0.1)
#RandomForestRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=100)
model_per.fit(X_train, y_train)

y_pred = model_per.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R2: {r2}')

0:	learn: 7.1400376	total: 1.16ms	remaining: 231ms
1:	learn: 6.8945656	total: 2.24ms	remaining: 222ms
2:	learn: 6.6889810	total: 3ms	remaining: 197ms
3:	learn: 6.5175791	total: 3.73ms	remaining: 183ms
4:	learn: 6.3671253	total: 4.74ms	remaining: 185ms
5:	learn: 6.2867275	total: 5.51ms	remaining: 178ms
6:	learn: 6.1777177	total: 6.29ms	remaining: 173ms
7:	learn: 6.0944791	total: 7.07ms	remaining: 170ms
8:	learn: 6.0244860	total: 7.93ms	remaining: 168ms
9:	learn: 5.9651623	total: 8.86ms	remaining: 168ms
10:	learn: 5.9284182	total: 9.69ms	remaining: 166ms
11:	learn: 5.8825703	total: 10.4ms	remaining: 164ms
12:	learn: 5.8405128	total: 11.2ms	remaining: 161ms
13:	learn: 5.8082553	total: 11.9ms	remaining: 158ms
14:	learn: 5.7825564	total: 12.6ms	remaining: 155ms
15:	learn: 5.7588254	total: 13.3ms	remaining: 153ms
16:	learn: 5.7420627	total: 14ms	remaining: 150ms
17:	learn: 5.7278487	total: 14.7ms	remaining: 148ms
18:	learn: 5.7116361	total: 15.4ms	remaining: 146ms
19:	learn: 5.7025420	total:

In [62]:
#WS
X = players_all_tr
y = players_all_nr['WS']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_ws = CatBoostRegressor(depth=3, iterations=200, learning_rate=0.1)
#RandomForestRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=100)
model_ws.fit(X_train, y_train)

y_pred = model_ws.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R2: {r2}')


0:	learn: 2.4438583	total: 829us	remaining: 165ms
1:	learn: 2.3117801	total: 1.95ms	remaining: 193ms
2:	learn: 2.2034442	total: 3.19ms	remaining: 209ms
3:	learn: 2.1075458	total: 3.99ms	remaining: 195ms
4:	learn: 2.0238275	total: 4.91ms	remaining: 191ms
5:	learn: 1.9526052	total: 6.61ms	remaining: 214ms
6:	learn: 1.8952471	total: 8.22ms	remaining: 227ms
7:	learn: 1.8434237	total: 9.52ms	remaining: 228ms
8:	learn: 1.8019221	total: 10.2ms	remaining: 216ms
9:	learn: 1.7677265	total: 11ms	remaining: 208ms
10:	learn: 1.7309735	total: 11.7ms	remaining: 200ms
11:	learn: 1.6999150	total: 12.3ms	remaining: 192ms
12:	learn: 1.6742612	total: 12.9ms	remaining: 186ms
13:	learn: 1.6546736	total: 13.6ms	remaining: 180ms
14:	learn: 1.6364739	total: 14.2ms	remaining: 175ms
15:	learn: 1.6208982	total: 14.9ms	remaining: 171ms
16:	learn: 1.6064151	total: 15.5ms	remaining: 167ms
17:	learn: 1.5935305	total: 16.2ms	remaining: 163ms
18:	learn: 1.5826293	total: 16.8ms	remaining: 160ms
19:	learn: 1.5732664	tota

In [63]:
#BPM
X = players_all_tr
y = players_all_nr['BPM']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_bpm = CatBoostRegressor(depth=3, iterations=200, learning_rate=0.1)
#RandomForestRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=100)
model_bpm.fit(X_train, y_train)

y_pred = model_bpm.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R2: {r2}')


0:	learn: 6.2404438	total: 986us	remaining: 196ms
1:	learn: 6.1221333	total: 2.33ms	remaining: 230ms
2:	learn: 6.0282051	total: 3.93ms	remaining: 258ms
3:	learn: 5.9525073	total: 4.7ms	remaining: 230ms
4:	learn: 5.8847899	total: 6.16ms	remaining: 240ms
5:	learn: 5.8450634	total: 7.07ms	remaining: 229ms
6:	learn: 5.7896265	total: 7.73ms	remaining: 213ms
7:	learn: 5.7527459	total: 8.35ms	remaining: 200ms
8:	learn: 5.7195893	total: 9.02ms	remaining: 191ms
9:	learn: 5.6907969	total: 9.73ms	remaining: 185ms
10:	learn: 5.6721786	total: 10.4ms	remaining: 178ms
11:	learn: 5.6519582	total: 11ms	remaining: 172ms
12:	learn: 5.6254884	total: 11.6ms	remaining: 167ms
13:	learn: 5.6103340	total: 12.2ms	remaining: 163ms
14:	learn: 5.5942813	total: 12.9ms	remaining: 159ms
15:	learn: 5.5838102	total: 13.7ms	remaining: 158ms
16:	learn: 5.5732152	total: 14.3ms	remaining: 154ms
17:	learn: 5.5661909	total: 15ms	remaining: 151ms
18:	learn: 5.5349846	total: 15.6ms	remaining: 149ms
19:	learn: 5.5292456	total: 

In [64]:
#VORP
X = players_all_tr
y = players_all_nr['VORP']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_vorp = CatBoostRegressor(depth=3, iterations=200, learning_rate=0.1)
#RandomForestRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=100)
model_vorp.fit(X_train, y_train)

y_pred = model_vorp.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R2: {r2}')


0:	learn: 1.0853857	total: 802us	remaining: 160ms
1:	learn: 1.0274829	total: 2.13ms	remaining: 211ms
2:	learn: 0.9739299	total: 3.64ms	remaining: 239ms
3:	learn: 0.9281653	total: 4.38ms	remaining: 215ms
4:	learn: 0.8886526	total: 5.01ms	remaining: 195ms
5:	learn: 0.8556830	total: 5.73ms	remaining: 185ms
6:	learn: 0.8278415	total: 6.36ms	remaining: 175ms
7:	learn: 0.8048751	total: 7.2ms	remaining: 173ms
8:	learn: 0.7847361	total: 7.91ms	remaining: 168ms
9:	learn: 0.7678463	total: 8.83ms	remaining: 168ms
10:	learn: 0.7548440	total: 9.58ms	remaining: 165ms
11:	learn: 0.7425740	total: 10.4ms	remaining: 164ms
12:	learn: 0.7327928	total: 11.2ms	remaining: 162ms
13:	learn: 0.7244147	total: 11.9ms	remaining: 158ms
14:	learn: 0.7192557	total: 12.5ms	remaining: 154ms
15:	learn: 0.7118001	total: 13.1ms	remaining: 151ms
16:	learn: 0.7067023	total: 13.8ms	remaining: 148ms
17:	learn: 0.7016127	total: 14.4ms	remaining: 145ms
18:	learn: 0.6972418	total: 15.1ms	remaining: 144ms
19:	learn: 0.6937991	tot

In [70]:
#now create a function that predicts all the stats for a player and a team

def predict_stats(player, team):
    rookie = rookies.loc[player]
    team = team_agg_22_23.loc[team]
    
    rookie_team = pd.concat([rookie, team])
    rookie_team = rookie_team.to_frame().T
    
    pts_pred = model_pts.predict(rookie_team)
    trb_pred = model_trb.predict(rookie_team)
    ast_pred = model_ast.predict(rookie_team)
    #per_pred = model_per.predict(rookie_team)
    ws_pred = model_ws.predict(rookie_team)
    #bpm_pred = model_bpm.predict(rookie_team)
    vorp_pred = model_vorp.predict(rookie_team)
    
    return pts_pred, trb_pred, ast_pred, ws_pred, vorp_pred

In [66]:
def predict_stats(player, team):
    rookie = rookies.loc[player]
    team = team_agg_22_23.loc[team]
    
    rookie_team = pd.concat([rookie, team])
    rookie_team = rookie_team.to_frame().T
    
    pts_pred = model_pts.predict(rookie_team)[0]
    trb_pred = model_trb.predict(rookie_team)[0]
    ast_pred = model_ast.predict(rookie_team)[0]
    ws_pred = model_ws.predict(rookie_team)[0]
    vorp_pred = model_vorp.predict(rookie_team)[0]
    
    return {
        'PTS': round(pts_pred, 2),
        'TRB': round(trb_pred, 2),
        'AST': round(ast_pred, 2),
        'WS': round(ws_pred, 2),
        'VORP': round(vorp_pred, 2)
    }

In [67]:
predict_stats('Paolo Banchero', 'SAS')

{'PTS': 13.92, 'TRB': 5.61, 'AST': 2.01, 'WS': 2.46, 'VORP': 0.41}

In [68]:
print("Walker Kessler stats for OKC: " + str(predict_stats('Walker Kessler', 'OKC')))
print("Walker Kessler stats for DEN: " + str(predict_stats('Walker Kessler', 'DEN')))

Walker Kessler stats for OKC: {'PTS': 13.34, 'TRB': 9.68, 'AST': 1.86, 'WS': 4.87, 'VORP': 0.98}
Walker Kessler stats for DEN: {'PTS': 12.41, 'TRB': 8.27, 'AST': 1.62, 'WS': 5.23, 'VORP': 1.01}


In [71]:
# Open the Dash app in a browser
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import webbrowser

import plotly.graph_objs as go

webbrowser.open('http://127.0.0.1:8050/')

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("Player Performance Prediction"),
    dcc.Dropdown(
        id='player-dropdown',
        options=[{'label': player, 'value': player} for player in rookies.index],
        value=rookies.index[0]
    ),
    dcc.Dropdown(
        id='team1-dropdown',
        options=[{'label': team, 'value': team} for team in team_agg_22_23.index],
        value=team_agg_22_23.index[0]
    ),
    dcc.Dropdown(
        id='team2-dropdown',
        options=[{'label': team, 'value': team} for team in team_agg_22_23.index],
        value=team_agg_22_23.index[1]
    ),
    dcc.Graph(id='prediction-graph')
])

# Define the callback to update the graph
@app.callback(
    Output('prediction-graph', 'figure'),
    [Input('player-dropdown', 'value'),
     Input('team1-dropdown', 'value'),
     Input('team2-dropdown', 'value')]
)
def update_graph(selected_player, selected_team1, selected_team2):
    pts_pred1, trb_pred1, ast_pred1, ws_pred1, vorp_pred1 = predict_stats(selected_player, selected_team1)
    pts_pred2, trb_pred2, ast_pred2, ws_pred2, vorp_pred2 = predict_stats(selected_player, selected_team2)
    
    data = [
        go.Bar(
            x=['PTS', 'TRB', 'AST', 'WS', 'VORP'],
            y=[pts_pred1[0], trb_pred1[0], ast_pred1[0], ws_pred1[0], vorp_pred1[0]],
            name=f'{selected_team1}',
            marker_color='blue'
        ),
        go.Bar(
            x=['PTS', 'TRB', 'AST', 'WS', 'VORP'],
            y=[pts_pred2[0], trb_pred2[0], ast_pred2[0], ws_pred2[0], vorp_pred2[0]],
            name=f'{selected_team2}',
            marker_color='red'
        )
    ]
    
    layout = go.Layout(
        title=f'Predicted Performance for {selected_player}',
        yaxis={'title': 'Value'},
        xaxis={'title': 'Metrics'},
        barmode='group'
    )
    
    return {'data': data, 'layout': layout}

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
